In [4]:
import pandas as pd
lines=pd.read_csv(r"English_to_Roman_Urdu - Sheet1.csv")

In [5]:
lines.head(20)

,eng,urdu
0,how is things?,cheezein kesi hain?
1,how is life?,zindagi kesi chal rahi?
2,what is happening?,kiya horaha>
3,what is going on?,kiya chalraha?
4,hey there!,arey haan
5,what is up?,aur kiya horaha hai?
6,good mornng,subah bakhair
7,good day,acha din
8,good afternoon,dopehr bakahir
9,good evening,sham bakhair


In [7]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

In [8]:
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.urdu=lines.urdu.apply(lambda x: x.lower())

# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.urdu=lines.urdu.apply(lambda x: re.sub("'", '', x))

exclude = set(string.punctuation) # Set of all special characters

# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.urdu=lines.urdu.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))

# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.urdu=lines.urdu.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.urdu=lines.urdu.apply(lambda x: re.sub(" +", " ", x))

# Add start and end tokens to target sequences
lines.urdu = lines.urdu.apply(lambda x : 'START_ '+ x + ' _END')

lines.sample(10)

,eng,urdu
65,have fun,START_ mazey karo _END
36,how is your day going,START_ app ka din kesa chal raha _END
107,sir how are you,START_ janab aap kesy han _END
63,hope you feel better,START_ umeed hai keh app behtar mehsoos karein...
53,its good to see you again,START_ tumhe dobara dekh ke acha laga _END
35,how is your day,START_ app ka din kesa hai _END
102,you are so humble,START_ tum bht aajiz hou _END
39,whats up,START_ kiya chal raha hai _END
3,what is going on,START_ kiya chalraha _END
100,do you have some place,START_ kiya aap ke paas koi jagah hai _END


In [10]:

# Vocabulary of English
all_eng_words=set()# Max Length of source sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
print('Max Source Length:',max_length_src)

# Max Length of target sequence
lenght_list=[]
for l in lines.urdu:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
print('Max Target Lenght:',max_length_tar)
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of Urdu 
all_urdu_words=set()
for urdu in lines.urdu:
    for word in urdu.split():
        if word not in all_urdu_words:
            all_urdu_words.add(word)# Max Length of source sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
print('Max Source Length:',max_length_src)

# Max Length of target sequence
lenght_list=[]
for l in lines.urdu:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
print('Max Target Lenght:',max_length_tar)

Max Source Length: 9
Max Target Lenght: 11
Max Source Length: 9
Max Target Lenght: 11


In [11]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_urdu_words))
num_encoder_tokens = len(all_eng_words)+1
num_decoder_tokens = len(all_urdu_words)+1
num_encoder_tokens, num_decoder_tokens

(121, 153)

In [12]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

154

In [13]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

lines = shuffle(lines)
lines.head(10)

,eng,urdu
10,good night,START_ shab bakhair _END
29,what is new,START_ naya kiya hai _END
97,happy to meet you,START_ aap se mil ke khushi hui _END
49,its been a while,START_ kaafi dair hwi _END
74,nice to meet you,START_ app se mil ke acha laga _END
78,how are you,START_ kese ho tum _END
80,good morning,START_ salaam alaikum _END
47,good luck,START_ achi qismat _END
7,good day,START_ acha din _END
15,how have you been,START_ app kese rahe _END


In [14]:
# Train - Test Split
X, y = lines.eng, lines.urdu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((99,), (12,))

In [15]:
#Save the train and test dataframes for reproducing the results later, as they are shuffled.

X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [16]:
def generate_batch(X = X_train, y = y_train, batch_size = 64):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [22]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 1
epochs = 50
latent_dim = 256
print(train_samples//batch_size)
print(val_samples//batch_size)

99
12


In [23]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [24]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [25]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [26]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [27]:
model.fit(generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Epoch 1/50
99/99 [==============================] - 16s 63ms/step - loss: 4.3253 - acc: 0.1968 - val_loss: 4.1098 - val_acc: 0.2745
Epoch 2/50
99/99 [==============================] - 3s 34ms/step - loss: 3.8596 - acc: 0.2209 - val_loss: 3.7063 - val_acc: 0.2941
Epoch 3/50
99/99 [==============================] - 3s 34ms/step - loss: 3.5216 - acc: 0.2430 - val_loss: 3.6326 - val_acc: 0.3137
Epoch 4/50
99/99 [==============================] - 3s 35ms/step - loss: 3.2105 - acc: 0.2831 - val_loss: 3.4806 - val_acc: 0.3333
Epoch 5/50
99/99 [==============================] - 3s 34ms/step - loss: 2.9338 - acc: 0.3273 - val_loss: 3.2646 - val_acc: 0.4510
Epoch 6/50
99/99 [==============================] - 3s 34ms/step - loss: 2.7196 - acc: 0.3594 - val_loss: 3.2450 - val_acc: 0.4510
Epoch 7/50
99/99 [==============================] - 3s 34ms/step - loss: 2.5202 - acc: 0.4016 - val_loss: 3.1999 - val_acc: 0.4314
Epoch 8/50
99/99 [==============================] - 3s 35ms/step - loss: 2.3318 - 

In [28]:
#Always remember to save the weights

model.save_weights('etu_weights.h5')

In [29]:
#Load the weights, if you close the application

model.load_weights('etu_weights.h5')

In [30]:
#Inference Setup

# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [31]:
#Decode sample sequeces

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [32]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [33]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Urdu Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Urdu Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 9ms/step
Input English sentence: how are you going
Actual Urdu Translation:  app kese ja rahe hain 
Predicted Urdu Translation:  app kese ja rahe hain 


In [34]:
from nltk.translate.bleu_score import sentence_bleu
reference = [
    y_train[k:k+1].values[0][6:-4].split()
]
print(reference)

[['app', 'kese', 'ja', 'rahe', 'hain']]


In [35]:
candidate = decoded_sentence[:-4].split()
print('BLEU score -> {}'.format(sentence_bleu(reference, candidate)))

BLEU score -> 1.0


In [36]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Urdu Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Urdu Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 9ms/step
Input English sentence: whats up
Actual Urdu Translation:  kiya chal raha hai 
Predicted Urdu Translation:  kiya chal raha hai 


In [37]:
from nltk.translate.bleu_score import sentence_bleu
reference = [
    y_train[k:k+1].values[0][6:-4].split()
]
print(reference)

[['kiya', 'chal', 'raha', 'hai']]


In [38]:
candidate = decoded_sentence[:-4].split()
print('BLEU score -> {}'.format(sentence_bleu(reference, candidate)))

BLEU score -> 1.0


In [39]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Urdu Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Urdu Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 11ms/step
Input English sentence: good night
Actual Urdu Translation:  shab bakhair 
Predicted Urdu Translation:  shab bakhair 


In [40]:
from nltk.translate.bleu_score import sentence_bleu
reference = [
    y_train[k:k+1].values[0][6:-4].split()
]
print(reference)

[['shab', 'bakhair']]


In [41]:
candidate = decoded_sentence[:-4].split()
print('BLEU score -> {}'.format(sentence_bleu(reference, candidate)))

BLEU score -> 1.491668146240062e-154


E:\anaconda\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
E:\anaconda\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [53]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [60]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Urdu Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Urdu Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 20ms/step
Input English sentence: i would love to travel again
Actual Urdu Translation:  mein dobara safar krna pasand krun ga 
Predicted Urdu Translation:  mai apko dobara dekh ke khush hun 


In [61]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Urdu Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Urdu Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 18ms/step
Input English sentence: good evening
Actual Urdu Translation:  shaam bakhair 
Predicted Urdu Translation:  sham bakhair 


In [62]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Urdu Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Urdu Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 23ms/step
Input English sentence: hi there
Actual Urdu Translation:  hi haan 
Predicted Urdu Translation:  hi 
